# Wykład 4 28.10.2020

## Definicje typów + Klasy typów i ich instancje + Moduły, importy, ogranizacja kodu źródłogego


### Type vs newtype vs data

In [ ]:
-- Typy w Haskellu
    -- typy proste: Int, Double, ...
    -- typy złożone: krotki, listy, funkcje, ...
    -- typy algebraiczne (created by ’algebraic’ operations): tworzone konstrukcją data ...
    

toUpper :: Char -> Char
toUpper c 
    | fromEnum(c)>=97 =  (toEnum(fromEnum c - 32))
    | otherwise = c

toLower :: Char -> Char
toLower c 
    | fromEnum(c)<97 =  (toEnum(fromEnum c + 32))
    | otherwise = c

type Name = String
capitalizeName :: Name -> Name
capitalizeName = map toUpper

:t capitalizeName -- capitalizeName :: Name -> Name

capitalizeName "t-1000" -- "T-1000"

In [ ]:
-- newtype !TYLKO JEDEN TYP!
newtype FirstName = FirstName String
formatFstName :: FirstName -> String
formatFstName (FirstName s) = case s of
    (x:xs) -> toUpper x : map toLower xs
    [] -> []

formatFstName(FirstName "apolinary")
-- formatFstName("apolinary") Couldn't match expected type ‘FirstName’ with actual type ‘String’

### Algebraiczne typy danych: product sum and sum types

In [ ]:
-- Product type [single constructor, ∼record]
data AandB a b = AandB_Con a b

data Person n s = Person n s -- *
:i Person -- data Person n s = Person n s 
:t Person -- Person :: forall n s. n -> s -> Person n s

p1 = Person "Inigo" "Montoya"
:t p1 -- p1 :: Person [Char] [Char]

### Sum type [co-product, disjoint union]

In [ ]:
data ABorB a b=AB_Con a b | B_Con b

data Either a b = Left a | Right b
:t Left -- Left :: forall a b. a -> Either a b

let r = Right 3
let l = Left 'a'
:t r -- r :: forall b a. Num b => Either a b

-- enum (special case of sum type)
data ThreeColors = Blue |
                    White |
                    Red
:t Blue -- Blue :: ThreeColors

### Record Syntax

In [ ]:
data Person = Person String String
    deriving (Show)
    
name :: Person -> String
name (Person n _) = n

surname :: Person -> String
surname (Person _ sn) = sn

let swordMaster = Person "Inigo" "Montoya"

-- Record syntax
data Person = Person
            { name :: String
            , surname :: String
            } 
let swordMaster = Person { surname = "Montoya" ,
    name = "Inigo" }
name swordMaster -- "Inigo"

### Parametric types, patterns in record syntax

In [ ]:
-- data Car = Car
--     { company :: String
--     , model :: String
--     , year :: Int
--     } deriving (Show)

-- cara1 = Car {company = "Hello", model = "It's me", year=1999}
-- :t cara1
-- :i cara1


data Car a b c = Car
    { company :: a
    , model :: b
    , year :: c
    } deriving (Show)

carInfo :: (Show a) => Car String String a -> String
carInfo Car{company = c, model = m, year = y} =
    "This " ++ c ++ " " ++ m ++ " was made in " ++ show y
    
let focus = Car {company="Ford", model="Focus I", year=2004}
carInfo focus

### Structural recursion (Rekursywne typy danych)

In [ ]:
data Tree a = Nil |
    Node a (Tree a) (Tree a) -- Recursion !IS HERE!
    deriving (Eq, Ord, Show, Read) -- Eq, Ord bo musza przynajmniej byc porownywalnymi, lub inne Tree

depth :: Tree a -> Int
depth Nil = 0
depth (Node n lt rt) = 1 + max (depth lt) (depth rt)

collapse :: Tree a -> [a]
collapse Nil = []
collapse (Node n lt rt) = collapse lt ++ [n] ++ collapse rt

mapTree :: (a -> b) -> Tree a -> Tree b
mapTree f Nil = Nil
mapTree f (Node n lt rt) = Node (f n) (mapTree f lt) (mapTree f rt)

n1 = Node 1
n2 = Node 2
t1 = (Node 1 Nil Nil) -- SHOULDN'T WRITE TREE in the begining!!!

### Typy wyższego rzędu (kinds) [ typy vs. konstruktory typów vs. konstruktory danych ]

In [ ]:
:i Int --
:k Int -- 
:i Maybe
:k Maybe
:k Maybe Int
:i Either
:k Either
:k Either Int

In [ ]:
data CDT a b =
    CDT { e :: Either a b
    , f :: Either a b -> Maybe a }
:k CDT
:t CDT
:t f

## Klasy typów w Haskellu vs. klasy w językach obiektowych


<b> Klasy typów w Haskellu to mechanizm realizacji polimorﬁzmu ad-hoc (’przeciążanie’ nazw). </b> <br> <br>
A class – collection of types (its instances) over which function(s) is (are deﬁned). One way we <br>
can think of a class is as an adjective: any particular type IS or IS NOT in the class. <br>
Other languages such as C++ or Java make a type and a class the same thing. <br>
A type is made a member or instance of a class by deﬁning the interface functions for the type.<br>
Instances in Haskell are global; it is not possible to make instances local to a module or a set of <br>
modules (note: ’wrapped’ type is often a solution to this problem).

In [ ]:
class [cx =>] <ClassName> tv where
-- signature (interface, contract) involving the type variable tv

instance [cx =>] <ClassName> <InsData> where -- e.g. InsData = Int
-- implementation of the methods declared in ClassName

### Klasy typów i ich instancje: przykład

In [ ]:
newtype Vec2Dnt a = Vec2Dnt (a, a)
instance Eq a => Eq (Vec2Dnt a) where
    (==) (Vec2Dnt (x1,y1)) (Vec2Dnt (x2,y2)) = x1 == x2 && y1 == y2
Vec2Dnt (2,2) == Vec2Dnt (2,2) -- True

In [ ]:
class VectorLike (t :: * -> *) where
    (|==|) :: Eq a => t a -> t a -> Bool
    (|+|), (|-|) :: (Num a) => t a -> t a -> t a
    (|*|) :: (Num a) => t a -> t a -> a
   
instance VectorLike Vec2Dnt where
    (|==|) (Vec2Dnt (x1,y1)) (Vec2Dnt (x2,y2)) = ...
    (|+|) (Vec2Dnt (x1,y1)) (Vec2Dnt (x2,y2)) = ...
    
(|-?) :: (VectorLike t, Num a, Eq a) => t a -> t a -> Bool
(|-?) v1 v2 = v1 |*| v2 == 0 -- is this 'safe' for all Num types?

## Moduły i importy

#### Program w Haskellu = kolekcja modułów (w tym Main zawierający funkcję main)

Rola/zastosowanie modułów <br>
tworzenie przestrzeni nazw (i zarządzanie nimi), mechanizm tworzenia abstrakcyjnych <br>
typów danych (ADT ), jednostka struktury kodu (jednostka kompilacji) <br>

In [ ]:
module MName (<exportNames>) where
...

import MName (<importNames>)
-- vs.
import MName hiding (<lst>)
-- vs.
import qualified MName as MyN
-- vs.
import qualified MName as NyN(x,y)

module Stack (Stack, push, pop) where
import Data.List
import Text.Printf(printf)
import qualified Data.Map as Map
import Prelude hiding (zip)
push :: a -> Stack a -> Stack a

module Stack (Stack(..),push,pop) --!

#### Stos jako ADT [przykład]

In [ ]:
module Stack (Stack, empty, isEmpty, push, top, pop) where
-- interface (signature, contract)
empty :: Stack a
isEmpty :: Stack a -> Bool
push :: a -> Stack a -> Stack a
top :: Stack a -> a
pop :: Stack a -> (a,Stack a)
-- implementation
newtype Stack a = StackImpl [a] -- hidden constructor
empty = StackImpl []
isEmpty (StackImpl s) = null s
push x (StackImpl s) = StackImpl (x:s)
top (StackImpl s) = head s
pop (StackImpl (s:ss)) = (s,StackImpl ss)